In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/events.csv')
df = df.dropna(subset=['itemid', 'transactionid'])
df['transactionid'] = df['transactionid'].astype(int)

In [3]:
df.sample(10)

,timestamp,visitorid,event,itemid,transactionid
2218271,1436370151351,519839,transaction,425359,5589
179622,1433875006348,568495,transaction,461686,2609
1999480,1432832909376,163561,transaction,284045,17528
218973,1434175525320,1396823,transaction,173291,5728
1592090,1431187099939,163561,transaction,120098,7629
2626908,1437928730534,54960,transaction,91105,15661
812683,1439407324025,861299,transaction,146325,4937
1529517,1430927389187,1161163,transaction,404864,7658
2023687,1432930961754,109357,transaction,14003,14786
630898,1438549871033,1007724,transaction,271812,5070


In [4]:
transactionItems = df.groupby('transactionid')['itemid'].agg(list)

In [5]:
def cartesian_set_product(items):
    results = []
    for a in items:
        for b in items:
            if a != b:
                results.append((a, b))
    return results

In [6]:
pairs = df.groupby('transactionid')['itemid'].agg(cartesian_set_product).sum()

In [7]:
def calculate_probability(item_pair):
    num_both = transactionItems.apply(lambda t: item_pair[0] in t and item_pair[1] in t).sum()
    num_given = transactionItems.apply(lambda t: item_pair[1] in t).sum()
    return num_both / num_given

In [8]:
probs = pd.Series(index=pd.MultiIndex.from_tuples(pairs, names=['itemid_of', 'itemid_given']))

In [9]:
for p in pairs:
    probs[p[0], p[1]] = calculate_probability(p)

In [10]:
probs.dropna().describe()

count    22580.000000
mean         0.662828
std          0.345664
min          0.007519
25%          0.333333
50%          0.666667
75%          1.000000
max          1.000000
dtype: float64